In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta, date
from sklearn.neural_network import MLPRegressor
import pickle

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
# Create DFTIME to Fill Empty Dates

def yearweek(date):
    date = pd.to_datetime(str(date))
    return str(date.isocalendar()[0]) + str(date.isocalendar()[1]).zfill(2)

def dftime(date, weeks):
    delta = timedelta(days = weeks*7)
    date = pd.to_datetime(str(date))
    date = date - delta
    dftime = pd.DataFrame(pd.period_range(start=date, periods=weeks, freq='7D', name = 'Date'))
    dftime['yearweek'] = dftime['Date'].apply(lambda x: yearweek(x))
    return dftime

In [17]:
# Connection data to Database

PATH = '/Users/renato/Downloads/Crystalball/'
DATABASE_MODEL = 'modelsample.db'
MODEL_CSV = 'model.csv'
DATABASE_HISTORY = 'purchasehistory.db'
DATABASE_CACHE = 'cache.db'
USER = 'user'
PASSWORD = 'password'
PORT = '5432'
HOST = '127.0.0.1'

In [15]:
# Default Data

CPF = '14566178854'

In [6]:
def get_data(cpf):
    try:
        connection = PATH + DATABASE_HISTORY
        sqliteConnection = sqlite3.connect(connection)
        cursor = sqliteConnection.cursor()
        sqlite_select_query = """SELECT * from history_purchase where cpf = """ + cpf
        cursor.execute(sqlite_select_query)
        records = cursor.fetchall()
        records = pd.DataFrame(records)
        records.columns = ['cpf', 'date1', 'prod_id', 'qtde']
        records['date'] =  pd.to_datetime(records['date1'],  format='%Y-%m-%d' )
        records = records.drop('date1', 1)

        cursor.close()
        
        return(records)

    except sqlite3.Error as error:
        print("Failed to read data from sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()

In [58]:
# Feature Engineering

def feature_engineering(df,date_forecast):

    
    #Load Model
    file = open(PATH+'MLP.dat', 'rb')
    model = pickle.load(file)
    file.close()
    
    
    # DataFrame for Loop
    dfloop = df.groupby(['prod_id'], as_index=False)['qtde'].count()
                
    
    #Iteract for product
    for i, r in dfloop.iterrows():
        # Filtrar Item_id em um novo DF
        dffilter = df[df['prod_id'] == r['prod_id']]

        #incluir campo ano/semana
        dffilter['yearweek'] = dffilter['date'].apply(lambda x: yearweek(x))
        #dffilter.set_index('yearweek', inplace = True)
        dfsoma = dffilter.groupby(['yearweek','prod_id'], as_index=False)['qtde'].sum()
        
        #Incluir dfmerge para semanas sem compras
        date = pd.to_datetime(str(date_forecast))
        dftempo = dftime(date, 8)
        dftempo.set_index('yearweek', inplace = True)

        # merge data
        dfsoma.set_index('yearweek', inplace = True)
        dfmerge = pd.concat([dfsoma,dftempo], axis=1)
        dfmerge['qtde'] = dfmerge['qtde'].fillna(0)
        print(dfmerge)
        
        #Colocar lags
        data = [dfmerge.iloc[-1,1], dfmerge.iloc[-2,1], dfmerge.iloc[-3,1], dfmerge.iloc[-4,1], dfmerge.iloc[-5,1], dfmerge.iloc[-6,1], dfmerge.iloc[-7,1], dfmerge.iloc[-8,1]]
        print(data)

        data = [data]
        result = model.predict(data)
        
        print(r['prod_id'])
        print(round(result[0],0))

In [60]:

df = get_data (CPF)
engdata = feature_engineering(df, '2020-04-20')
#retorno = save_data(engdata)
#retorno

prod_id  qtde        Date
201945      101     1         NaT
201946      101     1         NaT
201947      101     1         NaT
201948      101     1         NaT
201949      101     1         NaT
201950      101     1         NaT
201951      101     1         NaT
201952      101     1         NaT
202001      101     1         NaT
202002      101     1         NaT
202003      101     1         NaT
202004      101     1         NaT
202005      101     1         NaT
202006      101     1         NaT
202007      101     1         NaT
202008      101     1         NaT
202009      101     1  2020-02-24
202010      101     1  2020-03-02
202011      101     1  2020-03-09
202012      101     1  2020-03-16
202013      101     1  2020-03-23
202014      101     1  2020-03-30
202015      101     1  2020-04-06
202016      101     1  2020-04-13
[1, 1, 1, 1, 1, 1, 1, 1]
101
1.0
        prod_id  qtde        Date
201945    102.0   2.0         NaT
201947    102.0   2.0         NaT
201949    102.0   2.0  